# Basic tutorial on moving from previous iterations to TF2
## This assumes you are already using many of the older packages

# Importing data
## First, lets import a few tools that we will need, these include the Dicom_RT_Images_to_Mask which will read our data


In [ ]:
from Base_Deeplearning_Code.Dicom_RT_and_Images_to_Mask.Image_Array_And_Mask_From_Dicom_RT import Dicom_to_Imagestack, os
from Base_Deeplearning_Code.Plot_And_Scroll_Images.Plot_Scroll_Images import plot_scroll_Image, plot_Image_Scroll_Bar_Image

In [ ]:
dicom_image_path = r'\\mymdafiles\di_data1\Morfeus\BMAnderson\CNN\Data\Data_Liver\Miccai_Challenge\Images'
out_path = r'D:\Test'
description = 'Explanation'
excel_file = os.path.join('.','distribution.xlsx')

## Lets see what contours we have

In [ ]:
reader = Dicom_to_Imagestack(get_images_mask=False)
reader.down_folder(dicom_image_path)

In [ ]:
print('Print all rois that have something with "liver" in them')
for roi in reader.all_rois:
    if roi.lower().find('liver') != -1:
        print(roi)

In [ ]:
print('Lets just take the liver contours...')
reader.set_contour_names(['Liver'])

### Now, we've told the reader that we want every exam with a liver in it, if the contour does not exist, it will tell us

In [ ]:
reader.set_contour_names(['Liver_BMA_Program'])
reader.down_folder(dicom_image_path)


## Lets write the data out now

In [ ]:
print('Writing out data')
reader.set_contour_names(['Liver'])
reader.set_description()
reader.write_parallel(out_path=out_path, excel_file=excel_file)
print('Finished!')

### Now distribute them, either manually or with the distribute function into Train, Test, and Validation folders

In [ ]:
from Base_Deeplearning_Code.Dicom_RT_and_Images_to_Mask.Distribute_Train_Test_Validation import distribute
distribute(description=description,niftii_path=out_path, excel_file=excel_file)
print('Finished!')

# Turning data into tfrecord
## We will be training a 2D segmentation algorithm, so we want our training to be 2D slices
## But for validation we will want to predict on the entire image, so they will be 3D

In [ ]:
from Base_Deeplearning_Code.Make_Single_Images.Make_TFRecord_Class import write_tf_record, os
train_path = os.path.join(out_path,description,'Train')
test_path = os.path.join(out_path,description,'Test')
validation_path = os.path.join(out_path,description,'Validation')
write_tf_record(path=train_path,is_3D=False, extension=0)
write_tf_record(path=validation_path,is_3D=True)
write_tf_record(path=test_path,is_3D=True)
print('Finished writing records')

## Now our record files exist, along with a .pkl file that will be used to decode, and a Num_Examples so we can keep count

## Next lets write out generators

In [ ]:
from Base_Deeplearning_Code.Data_Generators.Generator_TF_Record import Data_Generator_Class
from Base_Deeplearning_Code.Data_Generators.Image_Processors_TF import *

### Load in the image processors that we want and the data generator class

In [ ]:
train_record = os.path.join(train_path,'Record_2D.tfrecord')
train_generator = Data_Generator_Class([train_record])
num_examples = len(train_generator)
print('The generator has {} examples'.format(len(train_generator)))

## Now we will load up image processors, this will convert our raw data into whatever we want

In [ ]:
print('Recommend always starting with these, it will decode and return images and annotations')

image_processors = [Decode_Images_Annotations(),
                    Decode_Bounding_Boxes_Volumes_Spacing(),
                    Return_Images_Annotations()]
train_generator.compile_data_set(image_processors=image_processors)

### Lets see what we got!

In [ ]:
data = next(iter(train_generator.data_set))
print(data)
print('As you can see, this is returning two 512x512 arrays, the first being our image, and the second being our mask')

### Lets add a few more processors to make this better...

In [ ]:
train_record = os.path.join(train_path,'Record_2D.tfrecord')
train_generator = Data_Generator_Class([train_record])
image_processors = [Decode_Images_Annotations(),
                    Decode_Bounding_Boxes_Volumes_Spacing(),
                    Return_Images_Annotations(),
                    Expand_Dimensions(axis=-1, on_images=True, on_annotations=True),
                    Ensure_Image_Proportions(tf.constant(512)),
                    Normalize_Images(mean_val=80, std_val=30),
                    Threshold_Images(lower_bound=-10, upper_bound=10),
                    {'cache'},
                    {'shuffle':len(train_generator)},
                    {'batch':10},
                    {'repeat'}]
train_generator.compile_data_set(image_processors=image_processors)

In [ ]:
print('The generator has {} examples'.format(len(train_generator)))
data = next(iter(train_generator.data_set))
# print(data)
print('As you can see, this is returning two 512x512 arrays, the first being our image, and the second being our mask')
images, annotation = data[0].numpy(), data[1].numpy()
annotation *= np.max(images)
data = np.concatenate([np.squeeze(images),np.squeeze(annotation)],axis=1)
plot_Image_Scroll_Bar_Image(data)

### Lastly, lets look at our validation data

In [ ]:
validation_record = os.path.join(validation_path,'Record_3D.tfrecord')
validation_generator = Data_Generator_Class([validation_record])
image_processors = [Decode_Images_Annotations(),
                    Decode_Bounding_Boxes_Volumes_Spacing(),
                    Return_Images_Annotations(),
                    Expand_Dimensions(axis=-1, on_images=True, on_annotations=True),
                    Ensure_Image_Proportions(tf.constant(512)),
                    Normalize_Images(mean_val=80, std_val=30),
                    Threshold_Images(lower_bound=-10, upper_bound=10),
                    {'cache'},
                    {'repeat'}]
validation_generator.compile_data_set(image_processors=image_processors)

In [ ]:
print('The generator has {} examples'.format(len(train_generator)))
data = next(iter(validation_generator.data_set))
# print(data)
print('As you can see, this is returning two 512x512 arrays, the first being our image, and the second being our mask')
images, annotation = data[0].numpy(), data[1].numpy()
annotation *= np.max(images)
data = np.concatenate([np.squeeze(images),np.squeeze(annotation)],axis=1)
plot_Image_Scroll_Bar_Image(data)

### Feel free to make any processor that you want, you can see the basic format in the top of Image_Processors_TF